In [51]:
# symbolic link
! ln -s /content/drive/My\ Drive/asm_coop /asm
% cd /asm/project_mvtec/

ln: failed to create symbolic link '/asm/asm_coop': Operation not supported
/content/drive/My Drive/asm_coop/project_mvtec


# Generate label txt files
Used https://github.com/ssaru/convert2Yolo to generate the labels file for both training and validation from json files

In [ ]:
# !pip3 list --outdated --format=freeze | grep -v '^\-e' | cut -d = -f 1  | xargs -n1 pip install -U

In [ ]:
% cd /asm/project_mvtec/convert2Yolo/
!pip3 install -r requirements.txt

In [ ]:
# generate training txt files
!python3 example.py --datasets COCO --img_path ../mvtec_d2s/images --label ../mvtec_d2s/annotations/D2S_training.json --convert_output_path ../mvtec_yolo/train --img_type ".jpg" --manifest_path ./ --cls_list_file ../mvtec_coco.names

In [ ]:
# generate validation txt files
!python3 example.py --datasets COCO --img_path ../mvtec_d2s/images --label ../mvtec_d2s/annotations/D2S_validation.json --convert_output_path ../mvtec_yolo/valid --img_type ".jpg" --manifest_path ./ --cls_list_file ../mvtec_coco.names

## Now mvtec_yolo/train and mvtec_yolo/valid contain only the label txt files

# Copy image files
copy the respective images from the mvtec_d2s dataset for the txt files in train and valid folders

In [61]:
from os import listdir
from shutil import copy

all_images_folder = "/asm/project_mvtec/mvtec_d2s/images/"
train_labels_folder = "/asm/project_mvtec/mvtec_yolo/train/"
valid_labels_folder = "/asm/project_mvtec/mvtec_yolo/valid/"

In [58]:
all_images = listdir(all_images_folder) 

In [59]:
train_labels = listdir(train_labels_folder)

In [60]:
valid_labels = listdir(valid_labels_folder)

In [ ]:
# copy train
for i in range(len(train_labels)) :
  txt = train_labels[i]
  img = txt.split('.')[0] + ".jpg"
  src = all_images_folder + img
  dst = train_labels_folder
  # print(src)
  copy(src, dst)

In [ ]:
# copy valid
for i in range(len(valid_labels)) :
  txt = valid_labels[i]
  img = txt.split('.')[0] + ".jpg"
  src = all_images_folder + img
  dst = valid_labels_folder
  # print(src)
  copy(src, dst)

In [22]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
img_name = "D2S_063421"
# img_loc = "/asm/darknet/data/mvtec_yolo/valid/"
img_loc = "/asm/project_mvtec/mvtec_d2s/images/"
full_address = img_loc + img_name + ".jpg"
imShow(full_address)

In [62]:
valid_labels = listdir(valid_labels_folder)

In [79]:
# split valid into valid and test
import random as r
r.seed(666)

valid_images = []
for f in valid_labels :
  if f.endswith('.jpg') :
    valid_images.append(f)
r.shuffle(valid_images)

new_valid = valid_images[:1800]
new_test = valid_images[1800:]

src = "/asm/project_mvtec/mvtec_yolo/valid/"
valid_dst = "/asm/project_mvtec/mvtec_yolo/valid_1800/"
test_dst = "/asm/project_mvtec/mvtec_yolo/test_1800/"

In [102]:
# copy the valid files to valid_1800
total = len(new_valid)
cur = 0
for img_file in new_valid :
  txt_file = img_file.split('.')[0] + '.txt'
  copy(src+img_file, valid_dst)
  copy(src+txt_file, valid_dst)
  cur += 1
  sys.stdout.write('\r')
  sys.stdout.write("%d out of %d copied" % (cur, total))

1800 out of 1800 copied

In [103]:
# copy the test files to test_1800
total = len(new_test)
cur = 0
for img_file in new_test :
  txt_file = img_file.split('.')[0] + '.txt'
  copy(src+img_file, test_dst)
  copy(src+txt_file, test_dst)
  cur += 1
  sys.stdout.write('\r')
  sys.stdout.write("%d out of %d copied" % (cur, total))

1800 out of 1800 copied

In [104]:
# generate train.txt
import os
train_folder = "/asm/project_mvtec/mvtec_yolo/train/"
image_files = []
os.chdir(train_folder)
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append(train_folder + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [105]:
# generate valid.txt
import os
train_folder = "/asm/project_mvtec/mvtec_yolo/valid_1800/"
image_files = []
os.chdir(train_folder)
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append(train_folder + filename)
os.chdir("..")
with open("valid.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [106]:
# generate test.txt
import os
train_folder = "/asm/project_mvtec/mvtec_yolo/test_1800/"
image_files = []
os.chdir(train_folder)
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append(train_folder + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

In [107]:
# Copy cfg to darknet folder 
% cd /asm/project_mvtec/yolo_config/
! cp obj.data /asm/darknet/data/
! cp yolov4-obj.cfg /asm/darknet/cfg/mvtec.cfg

/content/drive/My Drive/asm_coop/project_mvtec/yolo_config
